In [61]:
import open3d as o3d
import pandas as pd
import numpy as np

# azure
from pathlib import Path
import yaml
from azure.storage.blob import ContainerClient
from azure.core import exceptions
import re
from azure.servicebus import ServiceBusClient, ServiceBusMessage
from datetime import datetime
import os
import time

from annoy import AnnoyIndex

In [62]:

def download(connection_string, container_name, save_path,name_part=None, download_one=False):
    container_client = ContainerClient.from_connection_string(connection_string, container_name)
    labels=[]
    if not container_client.exists(): 
        print("wrong connection_string")
        return None
    else:
        blobs=container_client.list_blobs()
        for blob in blobs:
            blob_name=blob.name
            if (name_part is not None) and (blob_name!=name_part):continue
            filename=os.path.join(save_path,blob_name)
            if Path(filename).exists():
                labels.append(blob_name)
            else:
                StorageStreamDownloader = container_client.download_blob(blob)
                try:
                    file = open(filename, 'wb')
                except FileNotFoundError:
                    os.mkdir(save_path)
                    file = open(filename, 'wb')
                data=StorageStreamDownloader.readall()
                print(f"saving locally: {blob_name}")
                file.write(data)
                file.close()
                labels.append(blob_name)
            if download_one: break
    return labels

def load_config(dir_config):
    with open(os.path.join(dir_config, "config.yaml"), "r") as yamlfile:
        return yaml.load(yamlfile, Loader=yaml.FullLoader)

#score function
def get_score(annoyindex,points, print_stuff=False, vizualize_stuff=False, og_points=np.zeros((1,3))):
    NN_lines=np.zeros((len(points),2))
    NN_distances=np.zeros((len(points)))
    t_var=time.time()
    for NN_index in range(len(points)):
        index, dst = annoyindex.get_nns_by_vector(points[NN_index,:], 1, include_distances=True)
        if vizualize_stuff:NN_lines[NN_index,:]    =   [len(og_points)+NN_index,index[0]]
        NN_distances[NN_index]  =   dst[0]
    if print_stuff: print(f"took {time.time()-t_var} seconds to find nearest neighbors")
    score = np.sum(NN_distances/len(points))
    if vizualize_stuff:
        points=np.concatenate((og_points,points))
        colors = [[0,0,1] for i in NN_lines]
        lineset=o3d.geometry.LineSet(
            points=o3d.utility.Vector3dVector(points),
            lines=o3d.utility.Vector2iVector(NN_lines),
        )
        lineset.colors = o3d.utility.Vector3dVector(colors)
        return score, lineset
    return score


In [4]:
config=load_config("app")

## GET PCD AND CHECK 

In [63]:

container_name="containertest"
test_path = os.path.join("test")
labels=download(config["azure_storage_connectionstring"], container_name, test_path)

saving locally: 0ogpcdrotation.pcd
saving locally: 0ogpcdsource.pcd
saving locally: 0ogpcdtarget.pcd
saving locally: 0ogpcdtranslation.pcd
saving locally: 10ogpcdrotation.pcd
saving locally: 10ogpcdsource.pcd
saving locally: 10ogpcdtarget.pcd
saving locally: 10ogpcdtranslation.pcd
saving locally: 11ogpcdrotation.pcd
saving locally: 11ogpcdsource.pcd
saving locally: 11ogpcdtarget.pcd
saving locally: 11ogpcdtranslation.pcd
saving locally: 12ogpcdrotation.pcd
saving locally: 12ogpcdsource.pcd
saving locally: 12ogpcdtarget.pcd
saving locally: 12ogpcdtranslation.pcd
saving locally: 13ogpcdrotation.pcd
saving locally: 13ogpcdsource.pcd
saving locally: 13ogpcdtarget.pcd
saving locally: 13ogpcdtranslation.pcd
saving locally: 14ogpcdrotation.pcd
saving locally: 14ogpcdsource.pcd
saving locally: 14ogpcdtarget.pcd
saving locally: 14ogpcdtranslation.pcd
saving locally: 15ogpcdrotation.pcd
saving locally: 15ogpcdsource.pcd
saving locally: 15ogpcdtarget.pcd
saving locally: 15ogpcdtranslation.pcd
sav

In [66]:
pcd_src     =o3d.io.read_point_cloud(os.path.join(test_path, "12ogpcdsource.pcd"))
pcd_targ    =o3d.io.read_point_cloud(os.path.join(test_path, "12ogpcdtarget.pcd"))
pcd_rot     =o3d.io.read_point_cloud(os.path.join(test_path, "12ogpcdrotation.pcd"))
pcd_tran    =o3d.io.read_point_cloud(os.path.join(test_path, "12ogpcdtranslation.pcd"))

In [67]:
o3d.visualization.draw_geometries([pcd_src,pcd_targ])
o3d.visualization.draw_geometries([pcd_targ, pcd_tran])


In [38]:
# reg_p2p = o3d.pipelines.registration.registration_icp(
#     pcd_targ, pcd_tran, 0.01, np.eye(4),
#     o3d.pipelines.registration.TransformationEstimationPointToPoint(),
#     o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=1000))
# pcd_tran.transform(reg_p2p.transformation)
# print(reg_p2p)
o3d.visualization.draw_geometries([pcd_targ, pcd_tran])

create graph

In [68]:
t = AnnoyIndex(3, 'euclidean')
pcd_points=np.asarray(pcd_tran.points)

for i in range(len(pcd_points)):
    v = pcd_points[i,:]
    t.add_item(i, v)
t.build(10)


True

distance-
0.022724191734964656

In [69]:
pcd_scan_points=np.asarray(pcd_targ.points)
pcd_points=np.asarray(pcd_tran.points)
score,lineset=get_score(t,pcd_scan_points, vizualize_stuff=True, og_points=pcd_points)
print(score)

o3d.visualization.draw_geometries([pcd_tran,pcd_targ,lineset])

0.009555719628555361


### GET THE RESULTS AND PUT INTO ARRAY 

*shape: 16x24x24*

In [ ]:
container_name="containertest"
test_path = os.path.join("test")
labels=download(config["azure_storage_connectionstring"], container_name, test_path)